# Bank Customer Churn Analysis

## Project Overview
This project analyzes customer churn patterns in banking to identify at-risk customers and provide actionable retention strategies for financial institutions.

**Key Objectives:**
- Identify factors contributing to customer churn
- Analyze churn patterns across different customer segments
- Provide data-driven recommendations for customer retention

**Technologies:** Python, SQL (SQLite), Pandas, Matplotlib, Seaborn

## 1. Setup and Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ Libraries imported successfully")

## 2. Database Connection and Initial Data Load

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('data/bank_churn.db')
print("✓ Connected to database")

# Check available tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(f"\nAvailable tables: {tables['name'].tolist()}")

## 3. SQL Queries - Data Extraction

### 3.1 Overall Churn Statistics

In [ ]:
# Query: Overall churn rate
query_overall = """
SELECT 
    COUNT(*) as total_customers,
    SUM(CASE WHEN Exited = 1 THEN 1 ELSE 0 END) as churned_customers,
    ROUND(AVG(Exited) * 100, 2) as churn_rate_percent
FROM customers;
"""

overall_stats = pd.read_sql_query(query_overall, conn)
print("Overall Churn Statistics:")
print(overall_stats)

### 3.2 Churn Rate by Geography

In [ ]:
# Query: Churn rate by country
query_geography = """
SELECT 
    Geography,
    COUNT(*) as total_customers,
    SUM(Exited) as churned,
    ROUND(AVG(Exited) * 100, 2) as churn_rate
FROM customers
GROUP BY Geography
ORDER BY churn_rate DESC;
"""

churn_by_geo = pd.read_sql_query(query_geography, conn)
print("\nChurn Rate by Geography:")
print(churn_by_geo)

### 3.3 Churn by Credit Score Range

In [ ]:
# Query: Churn rate by credit score category
query_credit = """
SELECT 
    CASE 
        WHEN CreditScore < 600 THEN 'Poor (< 600)'
        WHEN CreditScore < 700 THEN 'Fair (600-699)'
        WHEN CreditScore < 800 THEN 'Good (700-799)'
        ELSE 'Excellent (800+)'
    END as credit_category,
    COUNT(*) as customers,
    ROUND(AVG(Exited) * 100, 2) as churn_rate,
    ROUND(AVG(CreditScore), 0) as avg_credit_score
FROM customers
GROUP BY credit_category
ORDER BY churn_rate DESC;
"""

churn_by_credit = pd.read_sql_query(query_credit, conn)
print("\nChurn Rate by Credit Score:")
print(churn_by_credit)

### 3.4 Active vs Inactive Members Analysis

In [ ]:
# Query: Active vs Inactive member comparison
query_activity = """
SELECT 
    CASE WHEN IsActiveMember = 1 THEN 'Active' ELSE 'Inactive' END as member_status,
    COUNT(*) as total_customers,
    ROUND(AVG(Exited) * 100, 2) as churn_rate,
    ROUND(AVG(Balance), 2) as avg_balance,
    ROUND(AVG(Tenure), 1) as avg_tenure_years
FROM customers
GROUP BY IsActiveMember
ORDER BY churn_rate DESC;
"""

churn_by_activity = pd.read_sql_query(query_activity, conn)
print("\nChurn Rate: Active vs Inactive Members:")
print(churn_by_activity)

### 3.5 Product Analysis

In [ ]:
# Query: Churn by number of products
query_products = """
SELECT 
    NumOfProducts,
    COUNT(*) as customers,
    ROUND(AVG(Exited) * 100, 2) as churn_rate,
    ROUND(AVG(Balance), 2) as avg_balance
FROM customers
GROUP BY NumOfProducts
ORDER BY NumOfProducts;
"""

churn_by_products = pd.read_sql_query(query_products, conn)
print("\nChurn Rate by Number of Products:")
print(churn_by_products)

### 3.6 Complaint Analysis

In [ ]:
# Query: Impact of complaints on churn
query_complain = """
SELECT 
    CASE WHEN Complain = 1 THEN 'Has Complained' ELSE 'No Complaints' END as complaint_status,
    COUNT(*) as customers,
    ROUND(AVG(Exited) * 100, 2) as churn_rate,
    ROUND(AVG([Satisfaction Score]), 2) as avg_satisfaction
FROM customers
GROUP BY Complain
ORDER BY churn_rate DESC;
"""

churn_by_complain = pd.read_sql_query(query_complain, conn)
print("\nChurn Rate by Complaint Status:")
print(churn_by_complain)

## 4. Load Full Dataset for Python Analysis

In [ ]:
# Load complete dataset
df = pd.read_sql_query("SELECT * FROM customers", conn)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()